## 10.4 Regression with Time Fixed Effects

- Controlling for variables that are constant across entities but vary over time can be done by including time fixed effects. 
- If there are only time fixed effects, the fixed effects regression model becomes $Y_{it} = \beta_0 + \beta_1 X_{it} + \delta_2 B2_t + \cdots + \delta_T BT_t + u_{it}$.
- This model eliminates omitted variable bias caused by excluding unobserved variables that evolve over time but are constant across entities.
- The entity and time fixed effects model is therefore

\begin{equation}
Y_{it} = \beta_0 + \beta_1 X_{it} + \gamma_2 D2_i + \cdots + \gamma_n DT_i + \delta_2 B2_t + \cdots + \delta_T BT_t + u_{it}
\end{equation}

- The combined model allows to eliminate bias from unobservables that change over time but are constant over entities and it controls for factors that differ across entities but are constant over time.
- The following code chunk shows how to estimate the combined entity and time fixed effects model of the relation between fatalities and beer tax,

\begin{equation}
FatalityRate_{it} = \beta_1 BeerTax_{it} + StateEffects + TimeFixedEffects + u_{it}
\end{equation}

- Like with the state fixed effects, we add the time fixed effects with the model's time variable encapsulated in fe().

In [1]:
using CSV
using DataFrames
using Query
using Plots
using FixedEffects
using FixedEffectModels
using CategoricalArrays
using LinearAlgebra

fatalities = CSV.read("/mnt/juliabox/Econometrics With Julia/Datasets/fatalities.csv")

fatalities.fatality_rate = fatalities.fatal ./ fatalities.pop * 10000
fatalities.state = categorical(fatalities.state)

fatalities_model = reg(
                        fatalities,
                        @formula(fatality_rate ~ beertax + fe(state) + fe(year))                        
)

println(fatalities_model)

x = fatalities.beertax
y = fatalities.fatality_rate

p1 = plot( #assign a plot object to the variable p1 using the following attributes
    x, #x series
    y, #y series
    st = :scatter, #series type
    title = "Traffic Fatality Rates and Beer Taxes from 1982-1988", #plot title
    label = "Observation", #legend labels
    xlabel = "Beer tax (in 1988 dollars)", #x axis label
    ylabel = "Fatality rate (fatalities per 10000)", #y axis label
    ylims = (0, 5), #y axis limits
    yticks = 0:0.5:5, #y axis tick range
    ms = 4, #marker size
    mc = :blue #marker color
)

y_prediction_diff(x) = dot(coef(fatalities_model), [1, x])

x = [minimum(fatalities.beertax), maximum(fatalities.beertax)]
y = y_prediction_diff.(x)

plot!(
    p1,
    x,
    y,
    st = :line,
    label = "OLS Difference"
)

display(p1)

MethodError: MethodError: no method matching length(::Expr)
Closest candidates are:
  length(!Matched::Core.SimpleVector) at essentials.jl:582
  length(!Matched::Base.MethodList) at reflection.jl:732
  length(!Matched::Core.MethodTable) at reflection.jl:806
  ...

- The estimated regression function is

\begin{align}
\widehat{FatalityRate} =  -\underset{(0.35)}{0.64} \times BeerTax + StateEffects + TimeFixedEffects
\end{align}